In [1]:
using AutomotiveDrivingModels
using AutoViz
using AutomotiveSensors
using AutomotivePOMDPs
using Parameters
using Reel
using StaticArrays
using ProfileView

using GridInterpolations 
using POMDPs
using POMDPToolbox
using QMDP
using JLD

using Revise

In [49]:
include("../src/pedestrian_crossing/pomdp_types.jl")
include("../src/pedestrian_crossing/spaces.jl")
include("../src/pedestrian_crossing/transition.jl")
include("../src/pedestrian_crossing/observation.jl")
include("../src/pedestrian_crossing/belief.jl")

include("../src/pedestrian_crossing/frenet_pedestrian_pomdp.jl")


animate_record (generic function with 2 methods)

In [40]:
function AutomotiveDrivingModels.observe!(model::FrenetPedestrianPOMDP, scene::Scene, roadway::Roadway, egoid::Int)

    model.t_current = model.t_current + model.timestep 
    model.tick += 1
    
    ego = scene[findfirst(scene, egoid)]
    model.ego_vehicle = ego
    model.sensor_observations = measure(model.sensor, ego, scene, roadway, model.obstacles)

    ################ High Level Planner ###################################################
    if ( model.tick % model.update_tick_high_level_planner == 0 )
        ego_y_state_space = ego_y_to_state_pace(pomdp, ego.state.posG.y)
        ego_v_state_space = ego_v_to_state_pace(pomdp, ego.state.v)
        
        println("--------------------------POMDP high level planner----------------------- t: ", model.t_current)
        println("x/y:", ego.state.posG.x, " / ",  ego.state.posG.y, " v:", ego.state.v)
        obs = pomdp.state_space[end]   

        for object in model.sensor_observations
          #  println(object)
            object_posF = Frenet(object.state.posG, get_lane(env.roadway, ego.state), env.roadway)

            delta_s = object_posF.s - ego.state.posF.s - ego.def.length/2
            delta_d = object_posF.t - ego.state.posF.t
            delta_theta = object_posF.ϕ - ego.state.posF.ϕ
            ped_v = object.state.v
            

            println("ds: ", delta_s, " dd: ", delta_d)
            
            obs = SingleOCFState(ego_y_state_space, ego_v_state_space, delta_s, delta_d, delta_theta, ped_v)
            println("obs: ", obs)
            obs_int = model.pomdp.state_space[state_index(pomdp,obs )]
            println("int: ", obs_int)           
 
        end
        
        pomdp.ego_vehicle = ego
        pomdp.obstacles = model.obstacles


        
        action_pomdp = SingleOCFAction(model.a.a_lat, model.a.a_lon)
        println(action_pomdp)
        b_ = update(model.updater, model.b, action_pomdp, obs)
        model.b = deepcopy(b_)
       # println("Belief new: ", model.b)
        act = action(model.policy, model.b) # policy
        println("policy: ", act)
    end
    
    model.risk = length(model.b)


    a_current = 0
    model.a = LatLonAccel(0.0, 0.0)
    
end


In [41]:
pomdp = SingleOCFPOMDP()
updater = SingleOCFUpdater(pomdp)
b0 = initial_state_distribution(pomdp)

solver = QMDPSolver(max_iterations=50, tolerance=1e-3) 

QMDP.QMDPSolver(50, 0.001, false)

In [28]:
@requirements_info solver pomdp
# run the solver
qmdp_policy = solve(solver, pomdp, verbose=true)
JLD.save("policy.jld", "policy", qmdp_policy)

09-Okt 11:08:05:WARNING:root:Keyword args for solve(::QMDPSolver, ::POMDP) are no longer supported. For verbose output, use the verbose option in the ValueIterationSolver



INFO: POMDPs.jl requirements for solve(::QMDPSolver, ::POMDPs.POMDP) and dependencies. ([✔] = implemented correctly; [X] = missing)

For solve(::QMDPSolver, ::POMDPs.POMDP):
  [No additional requirements]
For solve(::ValueIterationSolver, ::Union{POMDPs.MDP,POMDPs.POMDP}) (in solve(::QMDPSolver, ::POMDPs.POMDP)):
  [✔] discount(::SingleOCFPOMDP)
  [✔] n_states(::SingleOCFPOMDP)
  [✔] n_actions(::SingleOCFPOMDP)
  [✔] transition(::SingleOCFPOMDP, ::SingleOCFState, ::SingleOCFAction)
  [✔] reward(::SingleOCFPOMDP, ::SingleOCFState, ::SingleOCFAction, ::SingleOCFState)
  [✔] state_index(::SingleOCFPOMDP, ::SingleOCFState)
  [✔] action_index(::SingleOCFPOMDP, ::SingleOCFAction)
  [✔] actions(::SingleOCFPOMDP, ::SingleOCFState)
  [✔] iterator(::Array)
  [✔] iterator(::Array)
  [✔] iterator(::SparseCat)
  [✔] pdf(::SparseCat, ::SingleOCFState)
For ordered_states(::Union{POMDPs.MDP,POMDPs.POMDP}) (in solve(::ValueIterationSolver, ::Union{POMDPs.MDP,POMDPs.POMDP})):
  [✔] states(::SingleOCFPO

LoadError: [91mInterruptException:[39m

In [5]:
qmdp_policy = load("policy.jld")["policy"];

In [44]:
params = CrosswalkParams()
params.obstacles_visible = false

obstacle_1 = ConvexPolygon([VecE2(40, -1.5), VecE2(40, -4.5), VecE2(46.5, -4.5), VecE2(46.5, -1.5)],4)
obstacle_2 = ConvexPolygon([VecE2(40, +4.5), VecE2(40, +7.5), VecE2(46.5, +7.5), VecE2(46.5, +4.5)],4)
params.obstacles = [obstacle_1, obstacle_2]
#params.obstacles = [obstacle_2]

env = CrosswalkEnv(params)
pomdp.env = env


ego_v = 10.73
ped_v = 5.0/3.6
hitpoint = 150             # 50: middle, 100: left corner, 0: right corner
ped_theta = π/2


# fix values
ped_x = 50.0
ped_y_start = -2.5
ego_y = 0.0


ped_y_offset_hitpoint = VehicleDef().width * hitpoint / 100 - VehicleDef().width/2
ped_y_end = ped_y_offset_hitpoint
ped_t_collision = (ped_y_end - ped_y_start) / ped_v; 
ego_x = ped_x - ego_v * ped_t_collision - VehicleDef().length/2;

if ( false )
    ego_v = 0.0
    ego_x = 20
    ped_x = 30
    ped_y_start = -15.0
    ped_v = 0.0
end

# Car definition
car_initial_state = VehicleState(VecSE2(ego_x, ego_y, 0.), env.roadway.segments[1].lanes[1], env.roadway, ego_v)
car = Vehicle(car_initial_state, VehicleDef(), 1)

# Pedestrian definition using our new Vehicle definition
ped_initial_state = VehicleState(VecSE2(ped_x,ped_y_start,ped_theta), env.crosswalk, env.roadway, ped_v)
ped = Vehicle(ped_initial_state, AutomotivePOMDPs.PEDESTRIAN_DEF, 2)

scene = Scene()
push!(scene, car)
push!(scene, ped)



cam = FitToContentCamera(0.);
timestep = 0.05


pos_noise = 0.1
vel_noise = 0.1
false_positive_rate = 0.05
false_negative_rate = 0.01
rng = MersenneTwister(1);
sensor = AutomotiveSensors.GaussianSensor(AutomotiveSensors.LinearNoise(10, pos_noise, 0.00), 
                 AutomotiveSensors.LinearNoise(10, vel_noise, 0.00), false_positive_rate, false_negative_rate, rng) 



# define a model for each entities present in the scene
models = Dict{Int, DriverModel}()

ego_id = 1
ped_id = 2

models[ego_id] = FrenetPedestrianPOMDP(a=LatLonAccel(0.0, 0.0),env=env,sensor=sensor, obstacles=env.obstacles, timestep=timestep,
    env=env,
    pomdp=pomdp,
    policy=qmdp_policy,
    updater=updater,
    b=initial_state_distribution_ego_known(pomdp, ego_y_to_state_space(pomdp,ego_y), ego_v_to_state_space(pomdp,ego_v))
)

models[ped_id] = ConstantPedestrian(v_desired=ped_v, dawdling_amp=0.05) # dumb model

nticks = 80
rec = SceneRecord(nticks+1, timestep)

risk = Float64[]
sensor_observations = [Vehicle[]]
belief = SingleOCFBelief[]
ego_vehicle = Vehicle[]


obs_callback = (ObservationCallback(risk,sensor_observations, belief, ego_vehicle),)

tic()
simulate!(rec, scene, env.roadway, models, nticks, obs_callback)
toc()




--------------------------POMDP high level planner----------------------- t: 0.2
x/y:16.37441999999999 / 0.0 v:10.579999999999998
ds: 31.49882328011428 dd: -2.2258270828341904
obs: SingleOCFState(0.0, 10.76923076923077, 31.49882328011428, -2.2258270828341904, 1.5707963267948966, 1.8838437658955223)
int: SingleOCFState(0.0, 10.76923076923077, 32.333333333333336, -2.0, 1.57, 2.0)
SingleOCFAction(0.0, -1.0)
observation: SingleOCFState(0.0, 10.76923076923077, 31.49882328011428, -2.2258270828341904, 1.5707963267948966, 1.8838437658955223)
1000
2000
3000
4000
5000
6000
policy: SingleOCFAction(1.0, 1.0)
--------------------------POMDP high level planner----------------------- t: 0.39999999999999997
x/y:18.465419999999988 / 0.0 v:10.379999999999995
ds: 29.303212582258013 dd: -1.9804025246691719
obs: SingleOCFState(0.0, 10.76923076923077, 29.303212582258013, -1.9804025246691719, 1.5707963267948966, 1.1773518640461578)
int: SingleOCFState(0.0, 10.76923076923077, 30.25, -2.0, 1.57, 1.0)
SingleOCF

3000
4000
5000
6000
policy: SingleOCFAction(1.0, 1.0)
--------------------------POMDP high level planner----------------------- t: 2.9999999999999973
x/y:42.00841999999992 / 0.0 v:7.779999999999963
ds: 5.774188747021128 dd: 1.4069186527736346
obs: SingleOCFState(0.0, 7.538461538461538, 5.774188747021128, 1.4069186527736346, 1.5707963267948966, 1.7375537960293226)
int: SingleOCFState(0.0, 7.538461538461538, 5.25, 1.0, 1.57, 1.5)
SingleOCFAction(0.0, -1.0)
observation: SingleOCFState(0.0, 7.538461538461538, 5.774188747021128, 1.4069186527736346, 1.5707963267948966, 1.7375537960293226)
1000
2000
3000
4000
5000
6000
policy: SingleOCFAction(1.0, 1.0)
--------------------------POMDP high level planner----------------------- t: 3.1999999999999966
x/y:43.539419999999915 / 0.0 v:7.579999999999964
ds: 4.212759132749568 dd: 1.852865272355829
obs: SingleOCFState(0.0, 7.538461538461538, 4.212759132749568, 1.852865272355829, 1.5707963267948966, 1.3822920413424196)
int: SingleOCFState(0.0, 7.53846153

LoadError: [91mUndefVarError: a6 not defined[39m

In [45]:
duration, fps, render_hist = animate_record(rec, timestep, env, sensor, sensor_observations, risk, belief, ego_vehicle, CarFollowCamera(1,15.0))
film = roll(render_hist, fps = fps, duration = duration)

Reel.Frames{MIME{Symbol("image/png")}}("/tmp/tmpOUZsF3", 0x0000000000000050, 20.0, nothing)

In [9]:
function getObstructionCorner(obstacle::ConvexPolygon, ego_pos::VecE2)
 
    x = Vector{Float64}(obstacle.npts)
    y = Vector{Float64}(obstacle.npts)
    for i = 1:obstacle.npts
        x[i] = obstacle.pts[i].x
        y[i] = obstacle.pts[i].y
    end
    

    delta_s = maximum(x) - ego_pos.x
        
    right_side = true
    if ( ego_pos.y > mean(y) )
        delta_t = -(ego_pos.y -  maximum(y))
        right_side = true
    else
        delta_t = minimum(y) - ego_pos.y 
        right_side = false
    end
    return delta_s, delta_t, right_side 
end


getObstructionCorner (generic function with 1 method)

In [46]:
obstacle = models[ego_id].env.obstacles[2]
ego_pos = VecE2(10.0, -0.0) 
(ds, dt, right_side) = getObstructionCorner(obstacle, ego_pos)


LoadError: [91mBoundsError: attempt to access 0-element Array{AutomotiveDrivingModels.ConvexPolygon,1} at index [2][39m

In [55]:
@time v = ego_v_to_state_space_fast(pomdp, 12.0)
@time v = ego_v_to_state_space(pomdp, 12.0)

  0.000006 seconds (5 allocations: 176 bytes)
  0.000014 seconds (23 allocations: 1.094 KiB)


11.846153846153847